In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from sklearn.model_selection import train_test_split, KFold, PredefinedSplit, GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import matthews_corrcoef, balanced_accuracy_score, make_scorer
from imblearn.over_sampling import RandomOverSampler

from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier, AdaBoostClassifier
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier

%matplotlib inline

In [ ]:
import sys

file_path = "https://raw.githubusercontent.com/FarzinSohraby/PathInHydro/refs/heads/main/"

data = pd.read_csv(file_path+'Df_H2ase_CO_H2.csv')
data.shape

(899, 808)

In [ ]:
# Add a 'Molecule' column to distinguish CO and H2 based on the 'Trajectory' column
data['Molecule'] = data['Trajectory'].apply(lambda x: 'H2' if 'H2' in x else 'CO')
data = data.drop(["Trajectory", "Frame", "Pathway-new"], axis=1)
X = data.drop(['Binary', "Molecule"], axis=1)  # Features
y = data['Binary']  # Target variable

# Perform stratified train/validation/test split based on 'Molecule' column to ensure the 75/15/15% distribution for CO and H2
X_train, X_temp, y_train, y_temp = train_test_split(
    X, y, test_size=0.3, random_state=1, stratify=data['Molecule'])

# Now split the temporary set into validation and test sets
X_val, X_test, y_val, y_test = train_test_split(
    X_temp, y_temp, test_size=0.5, random_state=1, stratify=data.loc[y_temp.index, 'Molecule'])

# oversample the training set
oversampler = RandomOverSampler(random_state=1)
X_train_resampled, y_train_resampled = oversampler.fit_resample(X_train, y_train)

# Combine training and validation sets for GridSearchCV
X_combined = np.concatenate((X_train_resampled, X_val), axis=0)
y_combined = np.concatenate((y_train_resampled, y_val), axis=0)

# Create test_fold array for PredefinedSplit
# -1 for training data, 0 for validation data
test_fold = np.zeros(X_combined.shape[0], dtype=int)
test_fold[:len(X_train_resampled)] = -1  # Training data
test_fold[len(X_train_resampled):] = 0   # Validation data

ps = PredefinedSplit(test_fold=test_fold)

In [ ]:
# Calculate distribution percentages for the entire dataset
total_distribution = data['Molecule'].value_counts(normalize=True) * 100
print("Overall Distribution (%):\n", total_distribution)

# Calculate distribution in each set
train_distribution = data.loc[X_train.index, 'Molecule'].value_counts(normalize=True) * 100
val_distribution = data.loc[X_val.index, 'Molecule'].value_counts(normalize=True) * 100
test_distribution = data.loc[X_test.index, 'Molecule'].value_counts(normalize=True) * 100

print("\nTraining Set Distribution (%):\n", train_distribution)
print("\nValidation Set Distribution (%):\n", val_distribution)
print("\nTest Set Distribution (%):\n", test_distribution)


Overall Distribution (%):
 Molecule
CO    91.768632
H2     8.231368
Name: proportion, dtype: float64

Training Set Distribution (%):
 Molecule
CO    91.732909
H2     8.267091
Name: proportion, dtype: float64

Validation Set Distribution (%):
 Molecule
CO    91.851852
H2     8.148148
Name: proportion, dtype: float64

Test Set Distribution (%):
 Molecule
CO    91.851852
H2     8.148148
Name: proportion, dtype: float64


In [ ]:
# standardize the feature set for algorithms that requires normalization
scaler = StandardScaler().fit(np.array(X_train_resampled)) # fit the scaler with the training set
# transform the combined training & validation subset
X1_combined = scaler.transform(X_combined)

In [ ]:
# scoring metrics
scoring = {
    'BA': make_scorer(balanced_accuracy_score),
    'MCC': make_scorer(matthews_corrcoef)}

In [ ]:
# Define the SVM model
svm_classifier = SVC(random_state=1)

param_grid_svm = {
    'C': [0.1, 1, 10, 100],
    'kernel': ['linear', 'poly', 'rbf', 'sigmoid'],
    'gamma': ['scale', 'auto']
}

# Grid search
grid_search_svm = GridSearchCV(estimator=svm_classifier, param_grid=param_grid_svm,
                               cv=ps, scoring=scoring, refit=False, n_jobs=-1)
grid_search_svm.fit(X1_combined, y_combined) # standardized features

# Get the best parameters without refitting
best_params = grid_search_svm.cv_results_['params'][np.argmin(grid_search_svm.cv_results_['rank_test_BA'])]

# Output the best parameters
print("Best parameters:", best_params)

Best parameters: {'C': 100, 'gamma': 'scale', 'kernel': 'rbf'}


In [ ]:
log_reg_classifier = LogisticRegression(max_iter=5000, random_state=1)

param_grid_log_reg = {
    'C': [0.1, 1, 10, 100],
    'solver': ['newton-cg', 'lbfgs', 'liblinear', 'sag', 'saga'],
    'l1_ratio': [0, 0.25, 0.5, 0.75, 1]
}

# Grid search
grid_search_log_reg = GridSearchCV(estimator=log_reg_classifier, param_grid=param_grid_log_reg,
                                   cv=ps, n_jobs=-1, scoring=scoring, refit=False)
grid_search_log_reg.fit(X1_combined, y_combined) # standardized features

# Get the best parameters without refitting
best_params = grid_search_log_reg.cv_results_['params'][np.argmin(grid_search_log_reg.cv_results_['rank_test_BA'])]

# Output the best parameters
print("Best parameters:", best_params)

Best parameters: {'C': 10, 'l1_ratio': 0, 'solver': 'newton-cg'}


In [ ]:
# Define the KNN model
knn_classifier = KNeighborsClassifier()

# Set up the parameter grid for KNN
param_grid_knn = {
    'n_neighbors': [3, 5, 7, 9],
    'weights': ['uniform', 'distance'],
    'metric': ['euclidean', 'manhattan', 'minkowski']
}

# Grid search
grid_search_knn = GridSearchCV(estimator=knn_classifier, param_grid=param_grid_knn,
                               cv=ps, scoring=scoring, refit=False, n_jobs=-1)
grid_search_knn.fit(X_combined, y_combined) # unstandardized features

# Get the best parameters without refitting
best_params = grid_search_knn.cv_results_['params'][np.argmin(grid_search_knn.cv_results_['rank_test_BA'])]

# Output the best parameters
print("Best parameters:", best_params)

Best parameters: {'metric': 'manhattan', 'n_neighbors': 9, 'weights': 'uniform'}


In [ ]:
rf_classifier = RandomForestClassifier(n_estimators=500, oob_score=True, random_state=1)

param_grid_rf = {
    'max_depth': [10, 12, 15, 20, None],
    'ccp_alpha': [0, 0.0004, 0.0005, 0.0007, 0.001],
    'max_samples': [0.7, 0.8, 0.9, 0.95]
}

# Perform GridSearchCV with the predefined split and scoring
grid_search_rf = GridSearchCV(
    estimator=rf_classifier,
    param_grid=param_grid_rf,
    cv=ps,  # Using predefined split for cross-validation
    scoring=scoring,  # Use custom scoring metric(s)
    refit=False,  # Do not refit on the entire training data
    n_jobs=-1
)

# Fit the GridSearchCV model on the combined training set
grid_search_rf.fit(X_combined, y_combined)  # Using unstandardized features

# Get the best parameters without refitting
best_params = grid_search_rf.cv_results_['params'][np.argmin(grid_search_rf.cv_results_['rank_test_BA'])]

# Output the best parameters
print("Best parameters:", best_params)

Best parameters: {'ccp_alpha': 0, 'max_depth': 10, 'max_samples': 0.7}


In [ ]:
# Define the Gradient Boosting model
gb_classifier = GradientBoostingClassifier(n_estimators=500, tol=0.0005, random_state=1)

# Set up the parameter grid for Gradient Boosting
param_grid_gb = {
    'learning_rate': [0.005, 0.01, 0.025, 0.05, 0.1, 0.2],
    'max_depth': [3, 5, 7, 10],
    'subsample': [0.7, 0.8, 0.9]
}

# Grid search
grid_search_gb = GridSearchCV(estimator=gb_classifier, param_grid=param_grid_gb,
                              cv=ps, scoring=scoring, refit=False, n_jobs=-1)
grid_search_gb.fit(X_combined, y_combined) # unstandardized features

# Get the best parameters without refitting
best_params = grid_search_gb.cv_results_['params'][np.argmin(grid_search_gb.cv_results_['rank_test_BA'])]

# Output the best parameters
print("Best parameters:", best_params)

Best parameters: {'learning_rate': 0.01, 'max_depth': 3, 'subsample': 0.8}


In [ ]:
# Define the AdaBoost model
ada_classifier = AdaBoostClassifier(random_state=1)

# Set up the parameter grid for AdaBoost
param_grid_ada = {
    'n_estimators': [50, 100, 200, 500, 1000],
    'learning_rate': [0.01, 0.05, 0.1, 0.5, 1.0, 1.2]
}

# Grid search
grid_search_ada = GridSearchCV(estimator=ada_classifier, param_grid=param_grid_ada,
                               cv=ps, scoring=scoring, refit=False, n_jobs=-1)
grid_search_ada.fit(X_combined, y_combined) # unstandardized features

# Get the best parameters without refitting
best_params = grid_search_ada.cv_results_['params'][np.argmin(grid_search_ada.cv_results_['rank_test_BA'])]

# Output the best parameters
print("Best parameters:", best_params)

Best parameters: {'learning_rate': 0.05, 'n_estimators': 500}


In [ ]:
algorithms = ['SVM', 'Logistic Regression', 'kNN', 'Random Forest', 'Gradient Boost', 'AdaBoost']
gs_all = [grid_search_svm, grid_search_log_reg, grid_search_knn, grid_search_rf, grid_search_gb, grid_search_ada]

# Use 'np.argmin' to find the index of the best score for each GridSearchCV object
ba_all = [gs.cv_results_['mean_test_BA'][np.argmin(gs.cv_results_['rank_test_BA'])] for gs in gs_all]
mcc_all = [gs.cv_results_['mean_test_MCC'][np.argmin(gs.cv_results_['rank_test_BA'])] for gs in gs_all]

results = pd.DataFrame.from_dict({
    'algorithm': algorithms,
    'BA validation set': [f'{ba:.2f}' for ba in ba_all],
    'MCC validation set': [f'{mcc:.2f}' for mcc in mcc_all],
}, orient='columns')

display(results)

,algorithm,BA validation set,MCC validation set
0,SVM,0.89,0.71
1,Logistic Regression,0.90,0.76
2,kNN,0.86,0.63
3,Random Forest,0.96,0.86
4,Gradient Boost,0.96,0.86
5,AdaBoost,0.98,0.91


In [ ]:
# save/load the search results
import pickle

option = 'write'

if option == 'write':
    pickle_file = open(file_path+"algorithm_test_gsall_Binary.pkl", "wb")
    pickle.dump(gs_all, pickle_file)
else:
    pickle_file = open(file_path+"algorithm_test_gsall_Binary.pkl", "rb")
    [grid_search_svm, grid_search_log_reg, grid_search_knn, grid_search_rf, grid_search_gb, grid_search_ada] = pickle.load(pickle_file)
pickle_file.close()

In [ ]:
# check the details of selected search
search_results = grid_search_ada
check_metric = "BA" # "BA" or "MCC"

# retrieve the information
ba_scores = search_results.cv_results_[f'mean_test_{check_metric}']
hyperparam_combi = search_results.cv_results_['params']
# print results for each model in the search
for combi, ba in zip(hyperparam_combi, ba_scores):
    print(f"{str(combi):<65}         {ba:.3f}")

{'learning_rate': 0.01, 'n_estimators': 50}                               0.893
{'learning_rate': 0.01, 'n_estimators': 100}                              0.898
{'learning_rate': 0.01, 'n_estimators': 200}                              0.934
{'learning_rate': 0.01, 'n_estimators': 500}                              0.956
{'learning_rate': 0.01, 'n_estimators': 1000}                             0.960
{'learning_rate': 0.05, 'n_estimators': 50}                               0.934
{'learning_rate': 0.05, 'n_estimators': 100}                              0.956
{'learning_rate': 0.05, 'n_estimators': 200}                              0.960
{'learning_rate': 0.05, 'n_estimators': 500}                              0.982
{'learning_rate': 0.05, 'n_estimators': 1000}                             0.965
{'learning_rate': 0.1, 'n_estimators': 50}                                0.934
{'learning_rate': 0.1, 'n_estimators': 100}                               0.960
{'learning_rate': 0.1, 'n_estimators': 2